# Training MobileNet

In [ ]:
import os
from glob import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


X_train = np.load("/home/sajedhamdan/Desktop/skin_cancer/images_train_256x192.npy")
y_train = np.load("/home/sajedhamdan/Desktop/skin_cancer/train_labels.npy")

X_val = np.load("/home/sajedhamdan/Desktop/skin_cancer/images_val_256x192.npy")
y_val = np.load("/home/sajedhamdan/Desktop/skin_cancer/val_labels.npy")

X_train.shape, X_val.shape
y_train.shape, y_val.shape

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
# y_train.shape, y_val.shape


pretrained_model = MobileNet(input_shape=(192, 256, 3), include_top=False,weights='imagenet')

for layer in pretrained_model.layers:
    # print(layer.name)
    layer.trainable = False
    
# print(len(pretrained_model.layers))

In [ ]:
last_layer = pretrained_model.get_layer('conv_pw_13_relu')
print('Last layer output shape:', last_layer.output.shape)
last_output = last_layer.output


In [ ]:
x = layers.GlobalMaxPooling2D()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(7, activation='softmax')(x)

model = Model(pretrained_model.input, x)
#change epsilon from NONE to 1e-7, this helps not to divide by xero
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

train_datagenerator = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagenerator.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)


In [ ]:
batch_size = 64 
epochs = 3
history = model.fit(train_datagenerator.flow(X_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    validation_data=val_datagen.flow(X_val, y_val),
                    verbose=1,
                    steps_per_epoch=(X_train.shape[0] // batch_size),
                    validation_steps=(X_val.shape[0] // batch_size))                    


In [ ]:
for layer in model.layers[:60]:
    layer.trainable = False

for layer in model.layers[60:]:
    layer.trainable = True      

# reconfiguring and compling model
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])                      
              

In [ ]:
# decreasing learning rate when model is stuck(not improving)
lr_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=3)

batch_size = 64
epochs = 30
history = model.fit_generator(train_datagenerator.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size), callbacks=[lr_reduction])                                            


loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))                            

# Testing MobileNet

In [ ]:
import matplotlib.pyplot as plt

X_test = np.load("/home/sajedhamdan/Desktop/skin_cancer/images_test_256x192.npy")

y_test = np.load("/home/sajedhamdan/Desktop/skin_cancer/test_labels.npy")
y_test = to_categorical(y_test)

loss_test, accuracy_test = model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy_test, loss_test))

model.save("MobileNet.h5")

In [ ]:
# list of accuracy results on training and test data
acc = history.history['acc']
val_acc = history.history['val_acc']

# list of list results on training and test data
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

# plotting training and validation accuracy per epoch
plt.plot(epochs, acc, label = "training")
plt.plot(epochs, val_acc, label = "validation")
plt.legend(loc="upper left")
plt.title('Training and validation accuracy')

plt.figure()

# plotting training and validation loss per epoch
plt.plot(epochs, loss, label = "training")
plt.plot(epochs, val_loss, label = "validation")
plt.legend(loc="upper right")
plt.title('Training and validation loss')